In [ ]:
%matplotlib widget
from IPython.display import display, HTML
display(HTML("<style>.container{width:100% !important;}</style>"))

# Sampling Solutions with the Digital Annealer

As we already learned from the simulated annealing demo notebook, the Digital Annealer is a highly specialized sample hardware unit, which is able to perform a quantum-inspired simulated annealing. In most cases, the Digital Annealer is used to find an optimal solution for the underlying optimization problem. As expected, this is a single solution that is returned by the Digital Annealer. 

More precisely, when using the Digital Annealer, for a given optimization problem, several sampling hardware units are initialized and independently applied to find an optimal state, see figure below.
This increases the chance to find a very good solution for the problem. 

<img src="figures/sampling_pic_intro.PNG" width="500">


As a stochastic sampling unit with a high degree of parallelization we have the further ability to return several sub-optimal solutions of an optimization problem. 
Instead of using only the best solution found from the individual sampling units, this circumstance can be used to consider the solution of all units for a underlying problem. 

This is especially important if the Digital Annealer is used for a problem which can be understood as a specific Monte-Carlo simulation instead of a classical optimization task. Of course, the generated sub-optimal solutions can also be used for common optimization questions in order to perform deeper analyses of possible states. For example, sub-solutions can be filtered out for special properties that were not considered in the original optimization QUBO.

In this tutorial we will first have a brief discussion about the use of the Digital Annealer as sample unit. Then a Travelling-Salesman Problem (TSP) is introduced which afterwards will be used to get a better insight into the sampling tools available in the DADK python library. For a better demonstration we will use a prepared dashboard, that can be used as an experimental sandbox. 

## Monte Carlo Simulations with the Digital Annealer

The simulated annealing algorithm is a special case of the Metropolis algorithm which simulates a Markov process or chain of states based on a Metropolis-Hastings criterion with a Boltzmann distribution, see [Simulated Annealing & Metropolis Algorithm](https://pdfs.semanticscholar.org/3eea/67240fb0c2ec4f9ed5c61706c51dc770e102.pdf). The distribution of these Markov processes can not be usually simulated directly by some random variable.

In the classical simulated annealing algorithm, the Markov process is developed by cooling the temperature towards 0 in the Metropolis-Hastings criterion. At the final state of the Markov process, when the final annealing temperature is reached, hopefully a good (global/local) minimum of an underlying optimization function will be achieved. 

In many QUBO problems it is to be expected that the Digital Annealer will provide a different solution after each use as a solver than in a previous run. This may be due to the ambiguity of the solution (QUBO dependent), or to the never reached end temperature $"0"$, or to the randomly initialized initial states of the respective simulated Markov processes in the Digital Annealer. Ultimately, this can lead to a wide range of different solutions for a QUBO optimization problem. All these solutions represent statistical samples of the underlying Markov process, which is mostly determined by the given QUBO and initialized start random (Prior) distribution of the first state in the annealing algorithm.

It may now be of particular interest to consider QUBOs which do not contain an actual optimization goal. QUBOs that may consist of only constraints and whose purpose is to create states that fulfill these constraints. In this context, the solutions generated by the Digital Annealer may be of particular interest. The focus is now on the variety of solutions rather than the one optimal solution. 

### Example

One example, using simulated annealing for Monte Carlo simulations, is the work "Modeling of protein loops by simulated annealing" discussed by [Collura et al.(1993)](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2142460/pdf/8401234.pdf) in the journal "Protein Science" (Vol. 2, p.1502-1510, Cambridge  University  Press). They simulated loops of protein to be used in  homology modeling of proteins. The underlying target function was a combination of a "harmonic distance constraint" and a constrain describing "non-bonded atomic interactions". The underlying simulated annealing algorithm is used to obtain states that fulfill both constraints. Because of the stochastic character of the annealing algorithm described above, samples can be generated here. In the figure below we see two protein samples one can find in [Collura et al.(1993, figure 1)](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2142460/pdf/8401234.pdf).

<img src="figures/sampling_pic_example1.PNG" width="500">


A similar problem, using the Digital Annealer, is also considered by Fujitsu Laboratories.
The Digital Annealer can support an accelerated discovery of mid-size protein therapeutics used for drug discovery, see this [Webpage](https://www.fujitsu.com/global/about/resources/news/press-releases/2018/0918-01.html). There we can find the following: "... Fujitsu Laboratories, together with ProteinQure Inc., a company developing a software platform for protein drug discovery, confirmed the possibility of using the Digital Annealer on simulations that find stable configurations of protein drug candidates... By applying this technology to the second generation Digital Annealer, Fujitsu Laboratories was able to shorten the simulation time required for a computation involving a medium-sized drug candidate consisting of 48 amino acids (about 30 Kbits) from several hours using conventional computers to several minutes, using the same method of modeling amino acids..."

In the chart below we can see how Fujitsu Laboratories proceeded, see again [Webpage](https://www.fujitsu.com/global/about/resources/news/press-releases/2018/0918-01.html).

<img src="figures/Sampling_protein_annealer_example.jpg" width="500">
<font size="1"><center>Example application of the new technology to a stable configuration search problem</center></font>



## Test Problem - Travelling-Salesman (TSP)


The problem presented below serves us next as an experimental problem for explaining the sampling tools we can find in the Digital Annealer and DADK library.

By the Travelling-Salesman-Problem we understand the step-by-step determination of the visit of given cities for a round trip of a travelling salesman. The aim of the optimisation is to find the shortest route that is taken when visiting all cities. In the following we formulate this problem as QUBO, whose underlying optimization leads to an optimal binary decision variable, which we can interpret as the route of the round trip.

By the binary decision variable $x_{t,c}$ we describe that at time $t$, city $c$ will be visited where $c \in \lbrace 0,1,2,.., N-1 \rbrace$ and $t \in \lbrace 0,1,2,.., N-1 \rbrace$. 

The actual QUBO of the travelling salesman problem consists first of all of the constraint that at a certain time a travelling salesman visits exactly one city ...

$$
P_{\text{time}} = f_{\text{rules}} * \sum_{t=0}^{N - 1} \left(1 - \sum_{c=0}^{N - 1} x_{t,c} \right)^2
$$

Another constraint for ensuring that every city is visited only once is described by ...
$$
P_{\text{city}} = f_{\text{rules}} * \sum_{c=0}^{N - 1} \left(1 - \sum_{t=0}^{N - 1} x_{t,c} \right)^2
$$

The last term is the objective function. If the salesman goes through an edge, then we add up the cost involved in travelling that path. All the costs are then added up.
$$
P_{\text{distance}} = f_{\text{distance}} *  \sum_{t=0}^{N - 1} \sum_{c_0=0 \\c_1=0 \\c_0 \ne c_1}^{N - 1} \Delta_{c_0, c_1} x_{t,c_0} x_{(t+1) \% N, c_1} 
$$

$factor\_rules$, $factor\_distance$ are the penalty factors for the polynomials.
$$
    HQ = factor\_rules*P_{\text{time}} + factor\_rules*P_{\text{city}} + factor\_distance*P_{\text{distance}}
$$

 ## Dashboard Usage
Now that we have briefly discussed the Traveling-Salesman problem, we can begin to explore the pre-implemented dashboard, which serves as a sandbox for us to gain experience with the sampling tool of the Digital Annealer. In this regard, let us discuss the dashboard and optimizer class before we go the the Traveling-Salesman dashboard to explore the sampling mode. The dashboard discussion refers to the code cell to be activated below the heading "Dashboard in Sampling Mode". The simultaneously discussed optimizer class "Demo_Sampling_Model" can be found under the heading "TSP Class for Sampling:".

#### Setup Scenario
 
Already with a minimal understanding of geometric figures, the Traveling-Salesman problem used in this dashboard is very easy to solve. It is a finite set of cities to be visited, arranged on the boundary of a circle. Obviously the shortest route is a convex polygon which will approximate a circle for an increasing number of cities, see figure below. 


 
<img src="figures/parallel_temp_pic_Salesman.PNG" width="400">

In the tab **'Setup scenario'** one has the possibility to enter the number of cities used for the underlying Traveling-Salesman problem, here called **'Size'**, see figure below. Once the number of cities is set, the '**'Call'** button must be clicked and one can go to the **'Build QUBO'** tab in the dashboard.


<img src="figures/parallel_temp_pic_SetUpTab.PNG" width="800">

#### Build QUBO
 
In the **'Build QUBO'** environment it is possible to define the weights for the optimization target $P_{\text{distance}}$ with **'factor distance'** and the constraint terms $P_{\text{time}}$, $P_{\text{city}}$ with **'factor rules'**, see figure below. The weights must be carefully chosen so that all constraints are fulfilled and the optimization target is still relevant enough for the optimization.

Pressing the **'Default'** tabs automatically sets stored values that are set for this demonstration. Please press the **'Call'** button here and go to the **'Solve annealing'** tab.
 
 

<img src="figures/parallel_temp_pic_BuildTab.PNG" width="800">
 
 

#### Solve annealing
 
In the **'Solve annealing'** environment it is possible to set all necessary parameters to use sampling in the Digital Annealer. In the picture below we see in the **'Sampling'** tab all options that the Digital Annealer provides for sampling. 

<img src="figures/sampling_pic_solve_annealing.PNG" width="800">
<font size="1"><center>**'Solve annealing'** environment</center></font>

First of all there is the desired number of samples to be created. In this regard, the respective integer value must be entered to the right of the **'sample_size'** field. Since the number of DA units is limited, a maximum of 128 runs can be created with one call of the Digital Annealer. This can be set in the **'Annealing'** field with respect to **'number_runs'**. It should be noted that for large value settings for **'sample_size'**, the value **'number_runs'** should also be set to 128 thus the send and receive time is minimal. The DADK library automatically calls the Digital Annealer as often as necessary to obtain the desired sample size defined by **'sample_size'**.

You can set termination criteria for the sampling in the fields **'unique_sample_limit'** and **'filter_limit'**. 

**'unique_sample_limit'** is set as the maximum number of invalid solutions found during sampling. If the number is reached the sampling stops. Invalid solutions are determined by a filter. Such a filter is also used in the lower code cell where we define our TSP Optimizer. There we see the following: After building the TSP QUBO, the **'build_qubo'** method uses the list variable **'self.sample_filters'** which can be filled with filter criteria. A filter criterion can be set by the **'SampleFilter'** class, which is initialized by a QUBO (**'BinPol Class'**) with the **'qubo'** parameter and an upper limit float value defined in the **'upper_limit'** parameter. The polynomial has to be defined on the same search space as the polynomial used for sampling; typically it is a constraint part of a bigger polynomial, which is sampled. A given sample that is evaluated in such a QUBO filter and exceeds the upper limit is considered invalid. For the filter setting see the code snippet below:


``` python
    def build_qubo(self, factor_distance, factor_rules, silent=False):
        
        ...
        self.P_time = BinPol(self.qubo_vs)   
        ...
        self.P_city = BinPol(self.qubo_vs) 
        ...
        self.P_distance = BinPol(self.qubo_vs) 
        ...
        self.HQ = self.P_time + self.P_city + self.P_distance
        
        filter_time = SampleFilter(upper_limit = 0.01, qubo = self.P_time)
        filter_city = SampleFilter(upper_limit = 0.01, qubo = self.P_city)
        self.sample_filters = [filter_time, filter_city]

```

Let us note that if a filter is set only valid samples/solutions are stored in the solution list.


If the total frequency (i.e. number of occurrences as best solution of a run) of a certain valid sample reaches the value **'unique_sample_limit'** during sampling, the sampling will be aborted automatically. Here the valid samples are based on the previously discussed filter criteria. This is useful to stop e.g. samplings for simple energy functions with one global minimum, which otherwise could run endless delivering the same sample over and over again.

#### **prep_result** - Receive the samples for further processing

Now that we know how to build an optimizer class (see Optimizer_I Tutorials) we have to define a **'prep_result'** method in the optimizer.

To see and use the sampling modus the parameter **'solution_list'** must be used in the interface of **prep_result**. This parameter intercepts all generated solutions (samples) of the Digital Annealer.

``` python
    def prep_result(self, solution_list, silent=False):
        ...
```

In the code cell below we see the whole implementation of the Traveling Salesman problem as optimizer class **'Demo_Sampling_Model'**. Let us focus on the **'prep_result'** method where we can consider the possibility of using all Digital Annealer samples. 
In the **'prep_result'** method we obtain by the variable **'solution_list'** all solutions, i.e. the best solution and all sub-solutions. With the **'get_solution_list()'** we are able to get access to all solutions from the solution list, i.e. **'solution_list.get_solution_list()'**.
As discussed above, **'configuration = solution.configuration'** can be used to load the configuration of each solution defined as **'solution'** in the for loop. By the code line for the variable **'route'** the respective TS route of a sample is stored. In the variable **'distance'** we store the corresponding distance of this TS route.

``` python
        ...
        samples = []
        for solution in solution_list.get_solution_list():
            configuration = solution.configuration

            start_table = solution.extract_bit_array('x')
            route = [min(c if start_table.data[t][c] else self.N for c in range(self.N)) 
                     for t in range(self.N)]
            distance = sum(self.distance_table[route[t]][route[(t + 1) % self.N]] 
                           for t in range(self.N))

            samples.append({"solution":solution, "route":route, "distance":distance})
        ...
```


Furthermore we also find the attribute **'frequency'** which can be of special interest for sampling. With **'solution.frequency'**, you can get the number of occurrences of the respective solution for a TS routes. The Digital Annealer can, for example, determine similar solutions, some of which may be present in greater numbers. For example, if 100 solutions are sampled and a particular solution occurs 50 times, then this solution will only be stored once in the **'solution_list'** variable. This one item can then be read from the **'solution_list'** and the **'frequency'** is used to determine the actual multiplicity.

``` python
        ...
        for sample in samples:
            print("frequency=%d  route=%s  distance=%.2F" % 
                  (sample["solution"].frequency, str(sample["route"]), sample["distance"]))
        ...
```

Let us go back to the picture "**'Solve annealing'** environment" at the beginning of this chapter "Solve annealing".
Now that the **'Call'** button in the **'Solve annealing'** has been executed and all desired samples have been created, the print commands to be seen in the above code line are executed on the right, i.e. **'frequency'**, **'route'**, and the distance of the route with **'distance'**. 

#### Anneal tracker
 
In the **'Annealer tracker'** environment one can see the computing time of the Digital Annealer with more detail.
 

#### TSP Class for Sampling:

In the code cell below we find the TSP for sampling implemented as an optimizer. Please activate the cell and continue.

In [ ]:
import matplotlib.pyplot as plt

from random import randint 

from dadk.Optimizer import *

import networkx as nx

class Demo_Sampling_Model(OptimizerModel):
    
    ##################################################################################################################

    def __init__(self, persistent_file="Demo_Sampling.dao"):
        OptimizerModel.__init__(
            self,
            name = 'Demo_UIFunctions',
            load_parameter=[
                {'name':'N', 'type':'int_slider', 'value':12, 'min':2, 'max': 90, 
                 'label':'Size', 'description':'problem size'}                
            ],            
            build_qubo_parameter=[
                {'name':'factor_rules', 'type':'float_slider', 'value':100.0, 'min':0.0, 'max':10**6,
                 'label':'factor rules'},
                {'name':'factor_distance', 'type':'float_slider', 'value':1.0, 'min':0.0, 'max':10**3,
                 'label':'factor distance'}
            ], 
            #
            # Define default values for the "Solve annealing" tab.
            # The user can modify these fields, or press the button 
            # "Default" and get these values again.
            #
            default_solver_parameter={'CPU_number_iterations':1_000,
                                      'CPU_number_runs':1,
                                      'CPU_temperature_end':1,
                                      'CPU_offset_increase_rate':2.0,
                                      'CPU_solution_mode':'COMPLETE'},
            #
            # Define calculated values for the "Solve annealing" tab.
            # The user CAN'T modify these fields any more. The value
            # shown here is a default, which is will be overwritten with 
            # the methode self.Optimizer.set_calculated_solver_parameter(...)
            # see end of build_qubo for an example.
            #
            calculated_solver_parameter={'temperature_start': 8000},
            persistent_file=persistent_file
        )
    
    ##################################################################################################################
    def load(self, N=12, silent=False):
    ##################################################################################################################
        
        #Here we prepair all variables needed to build up the TSP QUBOs in the 'build_qubo' method
        self.N = N
        
        self.cities = [[100*math.sin(c*2*math.pi/N), 100*math.cos(c*2*math.pi/N)] for c in range(N)]
        
        self.distance_table = [[math.sqrt((self.cities[c_1][0] - self.cities[c_0][0])**2 + (self.cities[c_1][1] - self.cities[c_0][1])**2)
                                 for c_0 in range(N)] for c_1 in range(N)]
           
    ##################################################################################################################
    def build_qubo(self, factor_distance, factor_rules, silent=False):
    ##################################################################################################################

        #Here we set the QUBO of TSP
        self.qubo_vs = VarShapeSet(BitArrayShape('x', (self.N, self.N), axis_names=['Time', 'City']))        

        #--------------------------------------------------------------------------------

        self.P_time = BinPol(self.qubo_vs)   
        for t in range(self.N):
            self.P_time += (1 - BinPol.sum(Term(1, (('x', t, c),), self.qubo_vs) for c in range(self.N))) ** 2
        self.P_time *= factor_rules

        #--------------------------------------------------------------------------------

        self.P_city = BinPol(self.qubo_vs) 
        for c in range(self.N):
            self.P_city += (1 - BinPol.sum(Term(1, (('x', t, c),), self.qubo_vs) for t in range(self.N))) ** 2    
        self.P_city *= factor_rules        

        #--------------------------------------------------------------------------------

        self.P_distance = BinPol(self.qubo_vs) 
        for t in range(self.N):
            self.P_distance += BinPol.sum(Term(self.distance_table[c_0][c_1], (('x', t, c_0), ('x', (t + 1) % self.N, c_1),), self.qubo_vs) 
                                   for c_0 in range(self.N) for c_1 in range(self.N) if c_0 != c_1) 
        self.P_distance *= factor_distance

        #--------------------------------------------------------------------------------

        self.HQ = self.P_time + self.P_city + self.P_distance
        
        filter_time = SampleFilter(upper_limit = 0.01, qubo = self.P_time)
        filter_city = SampleFilter(upper_limit = 0.01, qubo = self.P_city)
        self.sample_filters = [filter_time, filter_city]

            
    ##################################################################################################################
    def prep_result(self, solution_list, silent=False):
    ##################################################################################################################
        #Here we present the sampling results obtained by the 'solution_list' which stores all solutions/samples
        samples = []
        for solution in solution_list.get_solution_list():
            configuration = solution.configuration

            start_table = solution.extract_bit_array('x')
            route = [min(c if start_table.data[t][c] else self.N for c in range(self.N)) for t in range(self.N)]
            distance = sum(self.distance_table[route[t]][route[(t + 1) % self.N]] for t in range(self.N))

            samples.append({"solution":solution, "route":route, "distance":distance})
            
        for sample in samples:
            print("frequency=%d  route=%s  distance=%.2F" % (sample["solution"].frequency, str(sample["route"]), sample["distance"]))


#### Dashboard in Sampling Mode
To activate the sampling mode in the Optimizer instantiation set the parameter **sampling=True**. This enabled the sampling parameters in the **Solve annealing** tab.
After activating the lower code cell you can try out the above described dashboard for sampling as a sandbox.

**This first cell is used as a example and can not be overwritten. In the second cell a dashboard is started that can be changed and those changes will be saved in the underlying 'dao' file.**

In [ ]:
optimizer_example = Optimizer(Demo_Sampling_Model('M_03_Sampling_example.dao'), sampling=True, auto_load=True, read_only=True) 

In [ ]:
optimizer = Optimizer(Demo_Sampling_Model('M_03_Sampling.dao'), sampling=True, auto_load=True) 